In [ ]:
#!pip install yt_dlp

   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ------ --------------------------------- 0.5/3.3 MB 3.4 MB/s eta 0:00:01
   ------------------------------- -------- 2.6/3.3 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 7.4 MB/s  0:00:00


In [ ]:
#!yt-dlp -f bestaudio --extract-audio --audio-format wav --write-subs --write-auto-subs --sub-langs "en,es" --convert-subs srt -o "%(title)s.%(ext)s" --exec "cmd /C if exist \"%(title)s.en.srt\" ren \"%(title)s.en.srt\" \"%(title)s_eng.srt\" & if exist \"%(title)s.es.srt\" ren \"%(title)s.es.srt\" \"%(title)s_es.srt\"" "https://www.youtube.com/watch?v=ParsL-0L38k"


[youtube] Extracting URL: https://www.youtube.com/watch?v=ParsL-0L38k
[youtube] ParsL-0L38k: Downloading webpage
[youtube] ParsL-0L38k: Downloading android sdkless player API JSON
[youtube] ParsL-0L38k: Downloading web safari player API JSON
[youtube] ParsL-0L38k: Downloading m3u8 information
[info] ParsL-0L38k: Downloading subtitles: en, es
[info] ParsL-0L38k: Downloading 1 format(s): 251
[info] Writing video subtitles to: ¿Qué significa ＂Quién pudiera＂ y cómo se usa？ ｜ Español directo al grano 🎯.en.vtt


ERROR: Unable to download video subtitles for 'en': HTTP Error 429: Too Many Requests


In [9]:
# scripts/download_dataset.py

import os
import re
from pathlib import Path
from typing import List, Dict, Optional

from yt_dlp import YoutubeDL


# Where your URL lists live
URL_LIST_DIR = Path("url_lists")

# Output dataset root
DATA_ROOT = Path("data")

# Mapping from logical language name -> (folder code, youtube subtitle code)
LANG_CONFIG: Dict[str, Dict[str, str]] = {
    "french":  {"code": "fr", "yt_sub": "fr"},
    "german":  {"code": "de", "yt_sub": "de"},
    "arabic":  {"code": "ar", "yt_sub": "ar"},
    "spanish": {"code": "es", "yt_sub": "es"},   # NOT "sp"!
    "chinese": {"code": "zh", "yt_sub": "zh"},
}


def read_urls(url_file: Path) -> List[str]:
    lines = []
    with url_file.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#"):
                lines.append(line)
    return lines


def make_lang_output_dir(lang_code: str) -> Path:
    out_dir = DATA_ROOT / lang_code
    out_dir.mkdir(parents=True, exist_ok=True)
    return out_dir


def srt_to_single_line_text(srt_path: Path, out_txt_name: str = "reference.txt") -> Optional[Path]:
    if not srt_path.exists():
        return None

    with srt_path.open("r", encoding="utf-8") as f:
        lines = f.readlines()

    text_lines = []
    ts_pattern = re.compile(
        r'^\s*\d{2}:\d{2}:\d{2},\d{3}\s*-->\s*\d{2}:\d{2}:\d{2},\d{3}'
    )

    for line in lines:
        # remove index lines
        if re.match(r'^\s*\d+\s*$', line):
            continue
        # remove timestamp lines
        if ts_pattern.match(line):
            continue
        stripped = line.strip()
        if not stripped:
            continue
        text_lines.append(stripped)

    one_line = " ".join(text_lines)
    out_path = srt_path.with_name(out_txt_name)
    with out_path.open("w", encoding="utf-8") as f:
        f.write(one_line + "\n")

    return out_path


def download_one_video(url: str, lang_code: str, yt_sub_lang: str):
    """
    Download best audio + subs (manual or auto) for a single video.
    Folder: data/<lang_code>/<video_title>/
    """
    lang_dir = make_lang_output_dir(lang_code)

    # Use yt-dlp to download as:
    # data/<lang_code>/<title>/<title>.<ext>
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": str(lang_dir / "%(title)s" / "%(title)s.%(ext)s"),
        "restrictfilenames": True,       # avoid crazy characters
        "noplaylist": True,

        # subtitles: manual + auto, specific language
        "writesubtitles": True,
        "writeautomaticsub": True,
        "subtitleslangs": [yt_sub_lang],
        "subtitlesformat": "srt",

        # Convert to WAV
        "keepvideo": False,
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "wav",
                "preferredquality": "0",
            }
        ],
    }

    with YoutubeDL(ydl_opts) as ydl:
        print(f"\n=== Downloading: {url} [{lang_code}] ===")
        info = ydl.extract_info(url, download=True)

        base_path = Path(ydl.prepare_filename(info))  # e.g. data/fr/Title/Title.webm
        folder = base_path.parent
        title_base = base_path.stem  # Title

        wav_path = folder / f"{title_base}.wav"
        if wav_path.exists():
            print(f"  Audio saved: {wav_path}")
        else:
            print("  WARNING: WAV not found – check ffmpeg installation.")

        # Find any .srt in that folder (for that language)
        srt_candidates = list(folder.glob("*.srt"))
        if not srt_candidates:
            print("  WARNING: no subtitles (.srt) found for this video.")
            return

        # Use the first srt (usually something like Title.fr.srt or Title.srt)
        srt_path = srt_candidates[0]
        print(f"  Subtitle file: {srt_path.name}")

        ref_txt = srt_to_single_line_text(srt_path)
        if ref_txt:
            print(f"  Cleaned reference written to: {ref_txt}")
        else:
            print("  Failed to write reference.txt")


def main():
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

    for lang_name, cfg in LANG_CONFIG.items():
        url_file = URL_LIST_DIR / f"{lang_name}.txt"
        if not url_file.exists():
            print(f"[INFO] URL file missing for {lang_name}: {url_file} (skipping)")
            continue

        urls = read_urls(url_file)
        if not urls:
            print(f"[INFO] No URLs in {url_file} (skipping)")
            continue

        lang_code = cfg["code"]
        yt_sub = cfg["yt_sub"]

        print(f"\n######## Processing language: {lang_name} ({lang_code}) ########")
        for url in urls:
            try:
                download_one_video(url, lang_code, yt_sub)
            except Exception as e:
                print(f"  ERROR processing {url}: {e}")


if __name__ == "__main__":
    main()



######## Processing language: french (fr) ########

=== Downloading: https://www.youtube.com/watch?v=e0JM62UGPF0 [fr] ===
[youtube] Extracting URL: https://www.youtube.com/watch?v=e0JM62UGPF0
[youtube] e0JM62UGPF0: Downloading webpage


[youtube] e0JM62UGPF0: Downloading android sdkless player API JSON
[youtube] e0JM62UGPF0: Downloading web safari player API JSON


[youtube] e0JM62UGPF0: Downloading m3u8 information


[info] e0JM62UGPF0: Downloading subtitles: fr
[info] e0JM62UGPF0: Downloading 1 format(s): 251
[info] Writing video subtitles to: data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..fr.srt


[download] Destination: data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..fr.srt
[download] 100% of   13.79KiB in 00:00:00 at 23.70KiB/s
[download] Sleeping 2.00 seconds as required by the site...
[download] Destination: data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..webm
[download] 100% of    8.40MiB in 00:00:12 at 669.43KiB/s 
[ExtractAudio] Destination: data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..wav
Deleting original file data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..webm (pass -k to keep)
  Audio saved: data\fr\Ne_dites_pas_J_ai_deux#\Ne_dites_pas_J_ai_deux..wav
  Subtitle file: Ne_dites_pas_J_ai_deux..fr.srt
  Cleaned reference written to: data\fr\Ne_dites_pas_J_ai_deux#\reference.txt

=== Downloading: https://www.youtube.com/watch?v=bDqNQEmEpB0 [fr] ===
[youtube] Extracting URL: https://www.youtube.com/watch?v=bDqNQEmEpB0
[youtube] bDqNQEmEpB0: Downloading webpage


...

[download] Destination: data\zh\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs.zh.srt
[download] 100% of   15.71KiB in 00:00:00 at 46.70KiB/s
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: data\zh\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs.webm
[download] 100% of   10.24MiB in 00:02:12 at 79.44KiB/s  
[ExtractAudio] Destination: data\zh\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs.wav
Deleting original file data\zh\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs.webm (pass -k to keep)
  Audio saved: data\zh\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs\Why_Chinese_Prefer_Big_Hospitals_Learn_Chinese_Through_Vlogs.wav
  Subtitle file: Wh

[youtube] uMrfGHnOQdY: Downloading android sdkless player API JSON
[youtube] uMrfGHnOQdY: Downloading web safari player API JSON


[youtube] uMrfGHnOQdY: Downloading m3u8 information


[info] uMrfGHnOQdY: Downloading subtitles: zh
[info] uMrfGHnOQdY: Downloading 1 format(s): 251
[info] Writing video subtitles to: data\zh\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs.zh.srt


[download] Destination: data\zh\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs.zh.srt
[download] 100% of   18.05KiB in 00:00:00 at 74.76KiB/s
[download] Sleeping 4.00 seconds as required by the site...
[download] Destination: data\zh\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs.webm
[download] 100% of   12.59MiB in 00:01:02 at 206.99KiB/s 
[ExtractAudio] Destination: data\zh\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs.wav
Deleting original file data\zh\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs\How_to_Survive_in_China_as_an_Introvert_or_Extrovert_Learn_Chinese_Through_Vlogs.webm (pass -k to keep)
  Audio 